In [1]:
import numpy as np
import pandas as pd
pd.options.display.float_format = '{:.8e}'.format

import configparser
from collections import defaultdict
from src.character import Character, CharacterType 
from src.stats import random_stats
from src.selection import stochastic_tournament_selection, deterministic_tournament_selection, add_relative_accumulate, elitist_selection, roulette_wheel_selection, universal_selection, boltzmann_selection
from src.genes import  encode_genes, decode_genes
from src.mutation import one_gene_mutation, multi_gene_mutation, multi_gene_mutation_uniform

In [2]:
import sys
sys.path.append("src")

from src.population import generate_init_population
from src.crossover import single_point_crossover, two_point_crossover, uniform_crossover, annular_crossover


In [ ]:
import sys
sys.path.append("src")

from src.population import generate_init_population
from src.character import CharacterType 
tipo1 = CharacterType.ARCHER
generation_0 = generate_init_population(10,150,CharacterType.DEFENDER)
generation_0

In [3]:
def read_config(filename):
    config = configparser.ConfigParser()
    config.read(filename)
    params = defaultdict(dict)
    for section in config.sections():
        for key, value in config.items(section):
            params[section][key] = value
    return params

params = read_config('config_file.config')
params

defaultdict(dict,
            {'CHARACTER': {'type': 'CharacterType.ARCHER'},
             'N': {'n': '200'},
             'M': {'m': '150'},
             'K': {'k': '120'},
             'GENERATIONS': {'number': '500'},
             'METHOD1PERCENTAGE': {'percentage': '0.4'},
             'SELECTION_METHOD': {'method1': "'elitist'",
              'method2': "'roulette_wheel'"},
             'BOLTZMANN': {'t_0': '50',
              't_offset': '10',
              'delta_t': '0.015',
              'm': '0.01'},
             'deterministic_tournament_selection': {'tournament_size': '50'},
             'stochastic_tournament_selection': {'threshold': '0.6'},
             'metodo_cruza': {'crossover': "'one_point'"},
             'MUTATION': {'mutation_type': "'multi_gene'",
              'mutation_rate': '0.7'},
             'METHODRePERCENTAGE': {'percentage': '0.6'},
             'metodo_reemplazo': {'metodo1': "'rank_based_selection'",
              'metodo2': "'deterministic_tournamen

In [4]:
from collections import namedtuple

def parameters():
    
    config_params = read_config('config_file.config')

    # Personaje 
    type = eval(config_params['CHARACTER']['type'])
    
    # sum( stats ) = 150
    maxStatsValue = eval(config_params['M']['m'])
    
    # N
    populationNumber = eval(config_params['N']['n'])
    # K
    k = eval(config_params['K']['k'])

    # Numero máximo de generaciones - criterio de corte
    numberOfGenerations = eval(config_params['GENERATIONS']['number'])
    
    # A%
    method1Percentage = eval(config_params['METHOD1PERCENTAGE']['percentage'])
    selectionMethod1 = eval(config_params['SELECTION_METHOD']['method1'])
    selectionMethod2 = eval(config_params['SELECTION_METHOD']['method2'])
   
    # CrossOver
    metodo_cruza = eval(config_params['metodo_cruza']['crossover'])
 
    # Mutation
    mutation_type = eval(config_params['MUTATION']['mutation_type'])
    mutation_rate = eval(config_params['MUTATION']['mutation_rate'])

    # B%
    methodReplacePercentage = eval(config_params['METHODRePERCENTAGE']['percentage'])
    metodo_reemplazo3 = eval(config_params['metodo_reemplazo']['metodo1'])
    metodo_reemplazo4 = eval(config_params['metodo_reemplazo']['metodo2'])

    cutCriteria = eval(config_params['CUT_CRITERIA']['criteria_type'])
    
    # Selector parameters
    # M
    deterministic_tournament_selection = eval(config_params['deterministic_tournament_selection']['tournament_size'])
    # stochastic_tournament_selection
    threhold = eval(config_params['stochastic_tournament_selection']['threshold'])
    # Boltzmann parameters
    boltzmannT_0 = eval(config_params['BOLTZMANN']['t_0'])
    boltzmannT_c = eval(config_params['BOLTZMANN']['t_offset'])
    boltzmannM = eval(config_params['BOLTZMANN']['m'])

    Param = namedtuple('Param',[
        'character',
        'maxStatsValue',
        'populationNumber',
        'k',
        'numberOfGenerations',
        'method1Percentage',
        'selectionMethod1',
        'selectionMethod2',
        'metodo_cruza',
        'mutation_type',
        'mutation_rate',
        'methodReplacePercentage',
        'metodo_reemplazo3',
        'metodo_reemplazo4',
        'cutCriteria',
        'deterministic_tournament_selection',
        'threhold',
        'boltzmannT_0',
        'boltzmannT_c',
        'boltzmannM'
    ])    

    parametros = Param(
                character=type,
                maxStatsValue=maxStatsValue,
                populationNumber=populationNumber,
                k=k,
                numberOfGenerations=numberOfGenerations,
                method1Percentage=method1Percentage,
                selectionMethod1=selectionMethod1,
                selectionMethod2=selectionMethod2,
                metodo_cruza=metodo_cruza,
                mutation_type=mutation_type,
                mutation_rate=mutation_rate,
                methodReplacePercentage=methodReplacePercentage,
                metodo_reemplazo3=metodo_reemplazo3,
                metodo_reemplazo4=metodo_reemplazo4,
                cutCriteria=cutCriteria,
                deterministic_tournament_selection=deterministic_tournament_selection,
                threhold=threhold,
                boltzmannT_0=boltzmannT_0,
                boltzmannT_c=boltzmannT_c,
                boltzmannM=boltzmannM
    )

    return parametros

p = parameters()
p.character

<CharacterType.ARCHER: 'ARCHER'>

In [5]:
tipo1 = CharacterType.ARCHER
generation_0 = generate_init_population(10,150,CharacterType.DEFENDER)
generation_0

Personaje creado: CharacterType.DEFENDER
Personaje creado: CharacterType.DEFENDER
Personaje creado: CharacterType.DEFENDER
Personaje creado: CharacterType.DEFENDER
Personaje creado: CharacterType.DEFENDER
Personaje creado: CharacterType.DEFENDER
Personaje creado: CharacterType.DEFENDER
Personaje creado: CharacterType.DEFENDER
Personaje creado: CharacterType.DEFENDER
Personaje creado: CharacterType.DEFENDER


,strength,agility,expertise,resistance,life,height,characterType,performance
0,65,4,50,16,15,1.90000000e+00,CharacterType.DEFENDER,None
1,4,121,6,1,18,1.90000000e+00,CharacterType.DEFENDER,None
2,2,99,5,7,37,1.50000000e+00,CharacterType.DEFENDER,None
3,33,40,13,17,47,1.70000000e+00,CharacterType.DEFENDER,None
4,81,10,16,2,41,1.70000000e+00,CharacterType.DEFENDER,None
5,70,24,7,37,12,1.80000000e+00,CharacterType.DEFENDER,None
6,37,20,29,61,3,1.60000000e+00,CharacterType.DEFENDER,None
7,16,7,74,12,41,1.30000000e+00,CharacterType.DEFENDER,None
8,75,9,41,10,15,1.80000000e+00,CharacterType.DEFENDER,None
9,63,8,16,60,3,1.60000000e+00,CharacterType.DEFENDER,None
